# Merge Ventilator data and HGHI State-level data

This notebook reads in the ventilator dataset and merges it with the state-level HGHI data.

From https://www.cambridge.org/core/journals/disaster-medicine-and-public-health-preparedness/article/mechanical-ventilators-in-us-acute-care-hospitals/F1FDBACA53531F2A150D6AD8E96F144D





In [17]:
import pandas as pd
import geopandas as gpd

from hscap.data import read_census_data_df

In [ ]:
vents_path = '../data/ventilators_by_state.csv'
vents_df = pd.read_csv(vents_path, encoding='utf-8')

In [ ]:
hghi_state_gdf = gpd.read_file('../data/hghi_state_data.geojson', encoding='utf-8')

In [ ]:
merged_df = hghi_state_gdf.set_index('State').join(vents_df.set_index('State Abbrv')).reset_index()
merged_df = merged_df.drop(columns=['Location'])


In [ ]:
merged_gdf = gpd.GeoDataFrame(merged_df, crs=4326)

## Add per-capita numbers

In [ ]:
census_df = read_census_data_df()
census2018_df = census_df[census_df['YEAR'] == 11]

In [27]:
state_pop_all = census2018_df[census2018_df['AGEGRP']==0].groupby(
    ['STNAME'])['TOT_POP'].sum().reset_index().rename({'STNAME': 'State Name'}, axis=1)

In [29]:
with_pop_gdf = merged_gdf.merge(state_pop_all, on='State Name')

In [31]:
with_pop_gdf.columns

Index(['State', 'State Name', 'Total Hospital Beds', 'Total ICU Beds',
       'Hospital Bed Occupancy Rate', 'ICU Bed Occupancy Rate',
       'Available Hospital Beds', 'Potentially Available Hospital Beds*',
       'Available ICU Beds', 'Potentially Available ICU Beds*',
       'Adult Population', 'Population 65+', 'Projected Infected Individuals',
       'Proejcted Hospitalized Individuals',
       'Projected Individuals Needing ICU Care',
       'Hospital Beds Needed, Six Months',
       'Percentage of Available Beds Needed, Six Months',
       'Percentage of Potentially Available Beds Needed, Six Months',
       'Percentage of Total Beds Needed, Six Months',
       'Hospital Beds Needed, Twelve Months',
       'Percentage of Available Beds Needed, Twelve Months',
       'Percentage of Potentially Available Beds Needed, Twelve Months',
       'Percentage of Total Beds Needed, Twelve Months',
       'Hospital Beds Needed, Eighteen Months',
       'Percentage of Available Beds Needed,

In [34]:
with_pop_gdf.dtypes

State                                                                                                   object
State Name                                                                                              object
Total Hospital Beds                                                                                     object
Total ICU Beds                                                                                          object
Hospital Bed Occupancy Rate                                                                            float64
ICU Bed Occupancy Rate                                                                                 float64
Available Hospital Beds                                                                                 object
Potentially Available Hospital Beds*                                                                    object
Available ICU Beds                                                                                      object
P

In [37]:
columns_to_calc_per_capita = [
    'Total Hospital Beds',
    'Total ICU Beds',
    'Available Hospital Beds',
    'Potentially Available Hospital Beds*',  
    'Available ICU Beds', 
    'Potentially Available ICU Beds*',
]

for c in columns_to_calc_per_capita:
    with_pop_gdf['{} per 100,000 people'.format(c)] = \
        with_pop_gdf[c].str.replace(',', '').astype('float32') / (with_pop_gdf['TOT_POP'] / 100000)
    

In [39]:
with_pop_gdf.to_file('../data/hghi_state_data_with_vents.geojson', 
                  encoding='utf-8', 
                  driver='GeoJSON')